# Collaborative filtering

## Item-to-item collaborative filtering

$$
         w(i, j) = \frac{\sum_{u\in U_i \cap U_j}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U_i \cap U_j} (r_{u,j}-\bar{r}_u)^2}}.
$$

\begin{equation}
 w(c,I_u) = \sum_{i\in I_u} w_{c,i}.
\end{equation}


### Import useful requirements

In [ ]:
import os

if not (os.path.exists("recsys.zip") or os.path.exists("recsys")):
    !wget https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
    !unzip recsys.zip

--2022-04-11 06:33:25--  https://github.com/nzhinusoftcm/review-on-collaborative-filtering/raw/master/recsys.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip [following]
--2022-04-11 06:33:25--  https://raw.githubusercontent.com/nzhinusoftcm/review-on-collaborative-filtering/master/recsys.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15312323 (15M) [application/zip]
Saving to: ‘recsys.zip’

recsys.zip          100%[===================>]  14.60M  --.-KB/s    in 0.1s    

2022-04-11 06:33:29 (98.9 MB/s) - ‘recsys.zip’ saved 

### Import requirements

In [ ]:
from recsys.datasets import ml1m, ml100k
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import os
import tqdm.notebook
import joblib
import sys

import typing as tp

### Dataset upload\

In [ ]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Will use movielens dataset

In [ ]:
ratings, movies = ml100k.load()

In [ ]:
ratings.head()

,userid,itemid,rating
0,1,1,5
1,1,2,3
2,1,3,4
3,1,4,3
4,1,5,3


In [ ]:
movies.head()

,itemid,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


### Preprocessing

In [ ]:
def ids_encoder(ratings):
    users = sorted(ratings['userid'].unique())
    items = sorted(ratings['itemid'].unique())

    # create users and items encoders
    uencoder = LabelEncoder()
    iencoder = LabelEncoder()

    # fit users and items ids to the corresponding encoder
    uencoder.fit(users)
    iencoder.fit(items)

    # encode userids and itemids
    ratings.userid = uencoder.transform(ratings.userid.tolist())
    ratings.itemid = iencoder.transform(ratings.itemid.tolist())

    return ratings, uencoder, iencoder

In [ ]:
# create the encoder
ratings, uencoder, iencoder = ids_encoder(ratings)

## Implementation

### Part 1. Similarities

\begin{equation}
 w_{i,j}= \frac{\sum_{u\in U}(r_{u,i}-\bar{r}_u)(r_{u,j}-\bar{r}_u)}{\sqrt{\sum_{u\in U} (r_{u,i}-\bar{r}_u)^2}\sqrt{\sum_{u\in U} (r_{u,j}-\bar{r}_u)^2}}.
\end{equation}

In [ ]:
def normalize(ratings: pd.DataFrame) -> pd.DataFrame:
    """
    Нормализует рейтинги по пользователям. Из каждого рейтинга вычитает средний рейтинг по пользователю.
    ratings: таблица рейтингов

    Возвращает:
        Таблица, содержащая все колонки таблицы `ratings` и колонку `norm_rating` с нормализованными рейтингами.
    """
    # calculate mean for every user
    mean = ratings.groupby(by='userid', as_index=False)['rating'].mean()
    norm_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='userid')

    # normalize each rating by substracting the mean rating of the corresponding user
    norm_ratings['norm_rating'] = norm_ratings['rating'] - norm_ratings['rating_mean']

    return norm_ratings[ratings.columns.tolist() + ['norm_rating']]

In [ ]:
def test_normalize():
    test_df = pd.DataFrame({
        "userid": [0, 0, 0, 1, 1],
        "itemid": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
    })

    expected = pd.DataFrame({
        "userid": [0, 0, 0, 1, 1],
        "itemid": [0, 1, 2, 1, 3],
        "rating": [2, 2, 5, 5, 5],
        "norm_rating": [-1, -1, 2, 0, 0]
    })

    assert test_df.shape[0] == expected.shape[0], "Number of user-item interactions is different"
    assert test_df.shape[1] + 1 == expected.shape[1], "Number of columns is incorrect"
    assert (normalize(test_df) == expected).all().all(), "Result is incorrect"

test_normalize()

In [ ]:
norm_ratings = normalize(ratings)
np_ratings = norm_ratings.to_numpy()
norm_ratings.head()

,userid,itemid,rating,norm_rating
0,0,0,5,1.389706
1,0,1,3,-0.610294
2,0,2,4,0.389706
3,0,3,3,-0.610294
4,0,4,3,-0.610294


In [ ]:
def cosine(x: np.array, y: np.array) -> float:
    """
    Функция, вычисляющая косинусное расстояние между векторами x и y.
    """
    if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
        return 0
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

In [ ]:
from functools import lru_cache

@lru_cache(2000)
def ratings_for_item(i):
    return np_ratings[np_ratings[:, 1] == i]

def calculate_similarity_between_two(np_ratings: np.array, i: int, j: int) -> float:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, rating_mean, norm_rating)
    i: номер первого айтема для вычисления похожести
    j: номер второго айтема для вычисления похожести

    Возвращает значение к-та корреляции Пирсона для айтемов i и j.
    """
    if i == j:
        return 1.0
    ratings_i, ratings_j = ratings_for_item(i), ratings_for_item(j)

    common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])
    common_ratings_i = ratings_i[np.isin(ratings_i[:, 0], common_users)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 0], common_users)]

    if len(common_users) > 0:
        assert sorted(common_ratings_i[:, 0]) == sorted(common_ratings_j[:, 0])
        x = common_ratings_i[:, 3]
        y = common_ratings_j[:, 3]
        return cosine(x, y)
    return -1.0

In [ ]:
assert np.isclose(calculate_similarity_between_two(np_ratings, 0, 0), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 2), 0.1069226)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 3), 0.0555092)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 5), -0.125509)
assert np.isclose(calculate_similarity_between_two(np_ratings, 1, 1431), 1.0)
assert np.isclose(calculate_similarity_between_two(np_ratings, 4, 1123), 0.0)

In [ ]:
def adjusted_cosine(np_ratings: np.array, similarity_between_two) -> tp.Tuple[np.array, np.array]:
    """
    Функция, вычисляющая к-т Пирсона для всевозможных пар айтемов (i, j).

    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, rating_mean, norm_rating)
    similarity_between_two: функция для подсчёта похожестей между двумя айтемами, принимает на вход массив с рейтингами и айди айтемов для подсчёта похожестей

    Возвращает:
        1. массив размера |I|x|I|, в i-ой строке которого расположены в порядке убывания похожести i-го айтема
        2. массив размера |I|x|I|, в i-ой строке которого расположены айди айтемов
            в порядке убывания их похожестей с i-ым айтемом
    """
  # создаём массив с нулями размера NxN, где N – количество айтемов
    nb_items = np.unique(np_ratings[:, 1]).size
    similarities = np.zeros(shape=(nb_items, nb_items))
    # заполняем его -1
    similarities.fill(-1)
    np.fill_diagonal(similarities, 1)

    # собираем массив с уникальными айдишками
    items = sorted(set(map(int, np_ratings[:, 1])))
    __import__('random').shuffle(items)

    with tqdm.notebook.tqdm(total=len(items) * (len(items) - 1) // 2) as pbar:
        for i in range(len(items)):
            for j in range(i + 1, len(items)):
                sim = similarity_between_two(np_ratings, items[i], items[j])
                similarities[items[i], items[j]] = sim
                similarities[items[j], items[i]] = sim
                pbar.update()
    assert np.all(similarities.T == similarities), 'Similarity matrix should be symmetrical'
    assert np.allclose(np.diag(similarities), 1.0), 'Similarities of items with themselves should be 1'

    # get neighbors by their neighbors in decreasing order of similarities
    # в каждой строке индексы ближайших соседей для данного айтема
    neighbors = np.flip(np.argsort(similarities), axis=1)

    # sort similarities in decreasing order
    # похожести для айтемов в том порядке, который указан в neighbors
    similarities = np.flip(np.sort(similarities), axis=1)

    return similarities, neighbors

In [ ]:
similarities, neighbors = adjusted_cosine(np_ratings, calculate_similarity_between_two)

  0%|          | 0/1413721 [00:00<?, ?it/s]

In [ ]:
assert np.equal(neighbors[1, :10],
            [ 988, 1473,  888, 1278,  902, 1287, 1377, 1402, 1367, 1431]).all()
assert np.equal(neighbors[2, :10],
            [1005, 1182, 1409, 1496, 1509, 1508, 1609, 1610, 1507, 1502]).all()
assert np.equal(neighbors[201, :10],
            [1553, 1172, 1464, 1681,  676,  813, 1481, 1490, 1491, 1497]).all()
assert np.equal(neighbors[800, :10],
            [ 641, 1422,  707,  891,  726, 1109,  887,  893,  892,  890]).all()

In [ ]:
def neighbours_viz(item_id: int, movies: pd.DataFrame,
                          similarities: np.array, neighbours: np.array, k=5):
    """
    item_id: id фильма, для которого вычисляются соседи
    movies: таблица с данными о фильмах
    similarities: массив похожестей
    neighbours: массив соседей для всех айтемов
    """
    film_name = movies[movies.itemid == iencoder.inverse_transform([item_id])[0]].title.values[0]
    similar_films = (
        (neighbor_id, movies[movies.itemid == iencoder.inverse_transform([neighbor_id])[0]].title.values[0], similarity)
        for neighbor_id, similarity in zip(neighbors[item_id][:k], similarities[item_id][:k])
    )
    display(pd.DataFrame(dict(zip(('item_id', film_name, 'Similarity'), zip(*similar_films)))))
    print('\n')

In [ ]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(914, movies, similarities, neighbors)
neighbours_viz(319, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,1638,Bitter Sugar (Azucar Amargo) (1996),1.0
1,1326,Captives (1994),1.0
2,1636,Girls Town (1996),1.0
3,1629,"Silence of the Palace, The (Saimt el Qusur) (1...",1.0
4,1242,Night Flier (1997),1.0


,item_id,Forrest Gump (1994),Similarity
0,765,Man of the Year (1995),1.0
1,1599,Guantanamera (1994),1.0
2,690,Dark City (1998),1.0
3,991,Head Above Water (1996),1.0
4,1606,Hurricane Streets (1998),1.0


,item_id,Primary Colors (1998),Similarity
0,1593,Everest (1998),1.0
1,622,Angels in the Outfield (1994),1.0
2,934,Paradise Road (1997),1.0
3,798,Boys Life (1995),1.0
4,266,unknown,1.0


,item_id,Paradise Lost: The Child Murders at Robin Hood Hills (1996),Similarity
0,1193,Once Were Warriors (1994),1.0
1,77,Free Willy (1993),1.0
2,1343,"Story of Xinghua, The (1993)",1.0
3,920,Farewell My Concubine (1993),1.0
4,1402,Caro Diario (Dear Diary) (1994),1.0


In [ ]:
def calculate_similarity_between_two_with_threshold(np_ratings: np.array, i: int, j: int) -> float:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, rating_mean, norm_rating)
    i: номер первого айтема для вычисления похожести
    j: номер второго айтема для вычисления похожести

    Возвращает значение к-та корреляции Пирсона для айтемов i и j.
    """
    if i == j:
        return 1.0
    ratings_i, ratings_j = ratings_for_item(i), ratings_for_item(j)
    common_users = np.intersect1d(ratings_i[:, 0], ratings_j[:, 0])
    common_ratings_i = ratings_i[np.isin(ratings_i[:, 0], common_users)]
    common_ratings_j = ratings_j[np.isin(ratings_j[:, 0], common_users)]

    if len(common_users) > 20:
        assert sorted(common_ratings_i[:, 0]) == sorted(common_ratings_j[:, 0])
        x = common_ratings_i[:, 3]
        y = common_ratings_j[:, 3]
        return cosine(x, y)
    return -1.0

In [ ]:
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 1, 1431), -1.0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 1, 17), -1.0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 4, 1123), -1.0)
assert np.isclose(calculate_similarity_between_two_with_threshold(np_ratings, 914, 1681), -1.0)

In [ ]:
similarities, neighbors = adjusted_cosine(np_ratings, calculate_similarity_between_two_with_threshold)

  0%|          | 0/1413721 [00:00<?, ?it/s]

In [ ]:
neighbours_viz(49, movies, similarities, neighbors)
neighbours_viz(68, movies, similarities, neighbors)
neighbours_viz(154, movies, similarities, neighbors)
neighbours_viz(200, movies, similarities, neighbors)

,item_id,Star Wars (1977),Similarity
0,49,Star Wars (1977),1.000000
1,171,"Empire Strikes Back, The (1980)",0.826287
2,180,Return of the Jedi (1983),0.728182
3,173,Raiders of the Lost Ark (1981),0.714250
4,407,"Close Shave, A (1995)",0.659379


,item_id,Forrest Gump (1994),Similarity
0,68,Forrest Gump (1994),1.000000
1,214,Field of Dreams (1989),0.445981
2,309,"Rainmaker, The (1997)",0.436435
3,21,Braveheart (1995),0.422572
4,965,"Affair to Remember, An (1957)",0.420235


,item_id,Dirty Dancing (1987),Similarity
0,154,Dirty Dancing (1987),1.000000
1,626,Robin Hood: Prince of Thieves (1991),0.727235
2,568,Wolf (1994),0.716361
3,254,My Best Friend's Wedding (1997),0.708146
4,416,"Parent Trap, The (1961)",0.634029


,item_id,Evil Dead II (1987),Similarity
0,200,Evil Dead II (1987),1.000000
1,183,Army of Darkness (1993),0.574317
2,23,Rumble in the Bronx (1995),0.536677
3,90,"Nightmare Before Christmas, The (1993)",0.491057
4,557,Heavenly Creatures (1994),0.481768


### Part 2. Top items for a user

In [ ]:
def candidate_items(np_ratings: np.array, userid: int, k=-1) -> tp.Tuple[np.array, np.array]:
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, rating_mean, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    k: количество кандидатов с каждого айтема из истории

    Возвращает
        1. массив I_u с id фильмов, просмотренных пользователем
        2. массив айтемов, близких к айтемам из истории пользователя
    """

    # 1. Finding the set I_u of items already rated by user userid
    I_u = np_ratings[np_ratings[:, 0] == userid]
    I_u = I_u[:, 1].astype('int')

    # 2. Taking the union of similar items for all items in I_u to form the set of candidate items
    c = set()

    for iid in I_u:
        # add the neighbors of item iid in the set of candidate items
        c.update(neighbors[iid, :k])

    c = list(c)
    # 3. exclude from the set C all items in I_u.
    candidates = np.setdiff1d(c, I_u, assume_unique=True)

    return I_u, candidates

In [ ]:
i_u, u_candidates = candidate_items(np_ratings, uencoder.transform([3])[0])

print('Количество просмотренных фильмов пользователя 1:', len(i_u))
print('Количество кандидатов для пользователя 1:', len(u_candidates))

Количество просмотренных фильмов пользователя 1: 54
Количество кандидатов для пользователя 1: 1628


In [ ]:
i_u_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([1])[0])
assert len(i_u_test) == 272
assert len(u_candidates_test) == 1410

i_u_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([50])[0])
assert len(i_u_test) == 24
assert len(u_candidates_test) == 1658

i_u_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([200])[0], 30)
assert len(i_u_test) == 216
assert len(u_candidates_test) == 593

i_u_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([200])[0], 15)
assert len(i_u_test) == 216
assert len(u_candidates_test) == 522

i_u_test, u_candidates_test = candidate_items(np_ratings, uencoder.transform([942])[0])
assert len(i_u_test) == 79
assert len(u_candidates_test) == 1603

del i_u_test, u_candidates_test

In [ ]:
def similarity_with_Iu(item_id: int, I_u: np.array) -> float:
    """
    item_id: id айтема-кандидата, для которого считается похожесть с историей пользователя
    I_u: массив id фильмов, просмотренных пользователем

    Возвращает число – похожесть айтема на историю пользователя.
    """
    w = 0
    for iid in I_u :
        # get similarity between itemid and c, if c is one of the k nearest neighbors of itemid
        if item_id in neighbors[iid] :
            w = w + similarities[iid, neighbors[iid] == item_id][0]
    return w

In [ ]:
i_u_test, _ = candidate_items(np_ratings, uencoder.transform([1])[0])
assert np.isclose(similarity_with_Iu(0, i_u_test), -18.147514)
assert np.isclose(similarity_with_Iu(200, i_u_test), -84.08526)

i_u_test, _ = candidate_items(np_ratings, uencoder.transform([300])[0])
assert np.isclose(similarity_with_Iu(200, i_u_test), -12.70523)
assert np.isclose(similarity_with_Iu(242, i_u_test), -0.99182)


In [ ]:
def rank_candidates(candidates: np.array, I_u: np.array) -> np.array:
    """
    candidates: массив id фильмов-кандидатов
    I_u: массив id фильмов, просмотренных пользователем

    Возвращает массив tuple, где первый элемент – айди айтема, второй – похожесть на историю пользователя
    """

    # list of candidate items mapped to their corresponding similarities to I_u
    sims = [similarity_with_Iu(c, I_u) for c in candidates]
    candidates = iencoder.inverse_transform(candidates)
    mapping = list(zip(candidates, sims))

    ranked_candidates = sorted(mapping, key=lambda couple:couple[1], reverse=True)
    return ranked_candidates

In [ ]:
i_u_test, candidates_test = candidate_items(np_ratings, uencoder.transform([1])[0])
assert len(rank_candidates(candidates_test, i_u_test)) == len(candidates_test)
assert rank_candidates(candidates_test, i_u_test)[0][0] == 313
assert rank_candidates(candidates_test, i_u_test)[10][0] == 302
assert rank_candidates(candidates_test, i_u_test)[19][0] == 603

## Putting it alltogether

In [ ]:
def topn_recommendation(np_ratings: np.array, userid, k=-1, N=30):
    """
    np_ratings: массив, каждый элемент которого является набором (user_id, item_id, rating, rating_mean, norm_rating)
    userid: id пользователя, для которого генерируются кандидаты
    k: количество кандидатов на стадии отбора кандидатов
    N: количество рекомендаций фильмов для пользователя

    Возвращает dataframe c рекомендацией top-N фильмов для пользователя userid.
    """
    # find candidate items
    I_u, candidates = candidate_items(np_ratings, userid, k)

    # rank candidate items according to their similarities with I_u
    ranked_candidates = rank_candidates(candidates, I_u)

    # get the first N row of ranked_candidates to build the top N recommendation list
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])
    topn = pd.merge(topn, movies, on='itemid', how='inner')
    return topn

In [ ]:
topn_recommendation(np_ratings, uencoder.transform([1])[0])

,itemid,similarity_with_Iu,title
0,313,-16.941690,Titanic (1997)
1,318,-23.278784,Schindler's List (1993)
2,655,-24.349682,Stand by Me (1986)
3,357,-26.289772,One Flew Over the Cuckoo's Nest (1975)
4,433,-27.140886,Heathers (1989)
5,423,-27.654851,E.T. the Extra-Terrestrial (1982)
6,651,-27.970798,Glory (1989)
7,288,-29.451573,Scream (1996)
8,276,-29.451749,Leaving Las Vegas (1995)
9,527,-29.573808,Gandhi (1982)


In [ ]:
test_history = [49, 81, 180, 256, 131, 379]
movies.iloc[test_history]

,itemid,title
49,50,Star Wars (1977)
81,82,Jurassic Park (1993)
180,181,Return of the Jedi (1983)
256,257,Men in Black (1997)
131,132,"Wizard of Oz, The (1939)"
379,380,Star Trek: Generations (1994)


In [ ]:
def candidate_items_by_user_history(I_u: tp.List[int], k=-1):
    c = set()
    for iid in I_u:
        c.update(neighbors[iid, :k])
    candidates = np.setdiff1d(list(c), I_u, assume_unique=True)

    return candidates

def topn_recommendations_by_user_history(I_u: tp.List[int], k=-1, N=30):
    candidates = candidate_items_by_user_history(I_u, k=k)
    ranked_candidates = rank_candidates(candidates, I_u)
    topn = pd.DataFrame(ranked_candidates[:N], columns=['itemid','similarity_with_Iu'])
    topn = pd.merge(topn, movies, on='itemid', how='inner')
    return topn

topn_recommendations_by_user_history(test_history)

,itemid,similarity_with_Iu,title
0,172,2.628615,"Empire Strikes Back, The (1980)"
1,174,2.142320,Raiders of the Lost Ark (1981)
2,313,2.035859,Titanic (1997)
3,210,1.941583,Indiana Jones and the Last Crusade (1989)
4,651,1.880540,Glory (1989)
5,22,1.773043,Braveheart (1995)
6,64,1.735550,"Shawshank Redemption, The (1994)"
7,79,1.707949,"Fugitive, The (1993)"
8,204,1.686313,Back to the Future (1985)
9,429,1.676720,"Day the Earth Stood Still, The (1951)"
